In [1]:
import sys
import django
import os
file_dir = "/Users/mirbilal/Desktop/minsir/"
if file_dir not in sys.path:
    sys.path.insert(0, file_dir)

os.environ["DJANGO_SETTINGS_MODULE"] = "minsirx.settings"
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true" 
os.environ["OPENAI_API_KEY"] = "sk-proj-FZ2v2sIScsxguqHnuT4VT3BlbkFJEgAuDrqMx91v7PG2fhDu"
django.setup()
from apps.email_manager.service_layer.email_linker import ConversationReader
import json
from datetime import datetime
from apps.email_manager.service_layer.insurance_erp_llm import InsuranceERPLLM
from apps.email_manager.models.email import EmailAttachment



/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/threadpoolctl.py:1214: RuntimeWarning: 
Found Intel OpenMP ('libiomp') and LLVM OpenMP ('libomp') loaded at
the same time. Both libraries are known to be incompatible and this
can cause random crashes or deadlocks on Linux when loaded in the
same Python program.
Using threadpoolctl may cause crashes or deadlocks. For more
information and possible workarounds, please see
    https://github.com/joblib/threadpoolctl/blob/master/multiple_openmp.md

  warnings.warn(msg, RuntimeWarning)


In [2]:
convo_reader = ConversationReader(
    start_date=datetime(year=2024, month=2, day=10),
    end_date=datetime(year=2024, month=2, day=20),
)
convo_reader.extract_conversation()


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-02-10 00:00:00) while time zone support is active.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField EmailData.date received a naive datetime (2024-02-20 00:00:00) while time zone support is active.
  warnings.warn(


In [3]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN ' in a_convo_str:
        (full_convo, attachments, all_files) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        insLLM = InsuranceERPLLM()
        insLLM.store_convo_in_vector_store(f"{full_convo}")
        insLLM.search_convo_vector_store()
        insLLM.query_from_final_results(insLLM.convo_query_results)
        print("------------------------------------")
        insLLM.store_documents_in_vector_store(attachments)
        insLLM.search_documents_vector_store()
        insLLM.query_from_final_results(insLLM.document_query_results)
        insLLM.clean_query_results()
        for attchmnt in attachments:
            print(attchmnt)
        # for a_msg in full_convo:
        #     print(json.dumps(a_msg, indent=4))
        insLLM.record_data(all_files)
        print("")
        print("------------------------------------")
        print("")
        break


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 0.3.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in

------------------------------------
/Users/mirbilal/Desktop/minsir/media/email_attachments/ADAMJEE_INSURANCE_PAYMENT_SWIFT_16.02.2024-2_ZrruCMA.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/PL-0123-202504-E12-000001_-_LAPTOPS_5PTg6tE.pdf

------------------------------------



/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField Policy.issue_date received a naive datetime (2024-02-16 00:00:00) while time zone support is active.
  warnings.warn(


In [8]:
from datetime import datetime
from os import environ
import re
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.chat_models import ChatOpenAI
from langchain.schema import (
    HumanMessage,
    SystemMessage,
    AIMessage
)
import json
import torch
from transformers import pipeline
from statistics import mode

from apps.risk_manager.models.policy import Policy, PolicyFile
from sqlalchemy.orm import sessionmaker
from sqlalchemy import create_engine

from db_tables import Base, SQLPolicyFile

In [10]:
print(environ.get("DB_HOST"))
DATABASE_URL = f'postgresql://{environ.get("POSTGRES_USER")}:{environ.get("POSTGRES_PASSWORD")}@{environ.get("DB_HOST")}:{environ.get("DB_PORT")}/{environ.get("POSTGRES_DB")}'

engine = create_engine(DATABASE_URL)

Session = sessionmaker(bind=engine)
session = Session()
if insLLM.event_type == 'policy_issued':
    new_policy = Policy.objects.create(
        issue_date=insLLM.queries_results["issue_date"]["value"],
        number=insLLM.queries_results["policy_number"]["value"],
        net_premium=insLLM.queries_results["net_premium"]["value"],
    )
    # /Users/mirbilal/Desktop/minsir/media/email_attachments/ADAMJEE_INSURANCE_PAYMENT_SWIFT_16.02.2024-2_ZrruCMA.pdf
    for file_path in attachments:
        final_path = file_path.replace("/Users/mirbilal/Desktop/minsir/media/", "")
        name = final_path.replace("email_attachments/", "")
        file = SQLPolicyFile(
            policy_id=new_policy.id,
            name = name,
            file = final_path
        )
        session.add(file)
    session.commit()

None


/Users/mirbilal/Desktop/minsir/minsirenv/lib/python3.12/site-packages/django/db/models/fields/__init__.py:1659: RuntimeWarning: DateTimeField Policy.issue_date received a naive datetime (2024-02-16 00:00:00) while time zone support is active.
  warnings.warn(


OperationalError: (psycopg2.OperationalError) could not translate host name "None" to address: nodename nor servname provided, or not known

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [6]:
# print(json.dumps(insLLM.convo_query_results, indent=4))
print("--------")
# print(json.dumps(insLLM.document_query_results, indent=4))
print("--------")
# print(json.dumps(insLLM.raw_results, indent=4))
# print("--------")
print(json.dumps(insLLM.queries_results, indent=4))

--------
--------


TypeError: Object of type datetime is not JSON serializable

In [4]:
print(insLLM.queries_results)

{'net_premium': {'value': 28437.0, 'type': 'double'}, 'issue_date': {'value': datetime.datetime(2024, 2, 16, 0, 0), 'type': 'datetime'}, 'covernote_number': {'value': None, 'type': 'string'}, 'policy_number': {'value': 'PL-0821-202504-E12-000024', 'type': 'string'}, 'claim_number': {'value': None, 'type': 'string'}, 'customer_name': {'value': 'SIKA PAKISTAN (PVT) LIMITED', 'type': 'string'}, 'risk_type': {'value': 'Electronic Equipment Insurance', 'type': 'string'}, 'event_type': {'value': 'policy_issued', 'type': 'string'}, 'sum_insured': {'value': 790499.0, 'type': 'double'}, 'premium_paid_amount': {'value': None, 'type': 'double'}, 'claim_intimation_amount': {'value': None, 'type': 'double'}, 'claim_paid_amount': {'value': None, 'type': 'double'}, 'premium_paid_date': {'value': None, 'type': 'datetime'}, 'claim_intimation_date': {'value': None, 'type': 'datetime'}, 'claim_paid_date': {'value': None, 'type': 'datetime'}}


In [11]:
from datetime import datetime
import re


for key, value_dict in insLLM.queries_results.items():
    data_type = value_dict["type"]
    data_value = value_dict["value"]
    # value_dict["value"] = insLLM.raw_results[key]
    # continue
    if data_value is not None and type(data_value) is str and "404 Not Found" not in data_value:
        # Clean the data value
        data_values = data_value.split("=")
        cleaned_value = data_values[-1].strip().replace("'", "")

        # Convert the cleaned data value to the specified type
        if data_type == "double":
            try:
                # Extract numbers from the string using regex
                cleaned_value = re.sub(r'[^\d.]', '', cleaned_value)
                cleaned_value = float(cleaned_value)
            except ValueError:
                cleaned_value = None
                raise Exception
        elif data_type == "datetime":
            try:
                # Extract date in the format YYYY-MM-DD using regex
                match = re.search(r'\d{4}-\d{2}-\d{2}', cleaned_value)
                if match:
                    cleaned_value = datetime.strptime(match.group(), "%Y-%m-%d")
                else:
                    cleaned_value = None
            except ValueError:
                cleaned_value = None
                raise Exception
        elif data_type == "string":
            cleaned_value = str(cleaned_value)
        
        # Update the value in the dictionary
        value_dict["value"] = cleaned_value

print(insLLM.queries_results)

{'net_premium': {'value': 28437.0, 'type': 'double'}, 'issue_date': {'value': datetime.datetime(2024, 2, 16, 0, 0), 'type': 'datetime'}, 'covernote_number': {'value': None, 'type': 'string'}, 'policy_number': {'value': 'PL-0821-202504-E12-000024', 'type': 'string'}, 'claim_number': {'value': None, 'type': 'string'}, 'customer_name': {'value': 'SIKA PAKISTAN (PVT) LIMITED', 'type': 'string'}, 'risk_type': {'value': 'Electronic Equipment Insurance', 'type': 'string'}, 'event_type': {'value': None, 'type': 'string'}, 'sum_insured': {'value': 790499.0, 'type': 'double'}, 'premium_paid_amount': {'value': None, 'type': 'double'}, 'claim_intimation_amount': {'value': None, 'type': 'double'}, 'claim_paid_amount': {'value': None, 'type': 'double'}, 'premium_paid_date': {'value': None, 'type': 'datetime'}, 'claim_intimation_date': {'value': None, 'type': 'datetime'}, 'claim_paid_date': {'value': None, 'type': 'datetime'}}


In [4]:
for a_convo_str, a_convo_data in convo_reader.conversations.items():
    if 'ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN ' in a_convo_str:
        (full_convo, attachments) = a_convo_data.get_full_convo()
        attachments: list[EmailAttachment] = attachments
        # insLLM = InsuranceERPLLM()
        # insLLM.store_documents_in_vector_store(attachments)
        # insLLM.search_documents_vector_store()
        # insLLM.query_from_final_results(insLLM.document_query_results)
        for attchmnt in attachments:
            print(attchmnt)
        for a_msg in full_convo:
            print(json.dumps(a_msg, indent=4))
        print("")
        print("------------------------------------")
        print("")
        break

/Users/mirbilal/Desktop/minsir/media/email_attachments/ADAMJEE_INSURANCE_PAYMENT_SWIFT_16.02.2024-2_ZrruCMA.pdf
/Users/mirbilal/Desktop/minsir/media/email_attachments/PL-0123-202504-E12-000001_-_LAPTOPS_5PTg6tE.pdf
{
    "from": "afzal.qadir@adamjeeinsurance.com",
    "to": [
        "mufzal.agha@adamjeeinsurance.com"
    ],
    "date": "February 19, 2024, 6:33 AM",
    "subject": "ISSUE POLICY FOR EEI | M/S SIKA PAKISTAN (PVT.) LTD.",
    "body": "Dear Agha,\n\u00a0\nPlease issue policy for EEI as per payment instrument attached.\n\u00a0\n\u00a0 Afzal Qadir\nTeam Member,\nMiscellaneous Underwriting\nP\u00a0\n:\n\u00a0\u00a0+92 21 32412623\u00a0\n\u00a0Ext \n1370 , \nM\u00a0\n: +92 3350203993\u00a0\u00a0\nE\u00a0\n:\u00a0\nafzal.qadir\n@adamjeeinsurance.com\nAdamjee Insurance Company Limited\nAdamjee House Division, 5th Floor, \nAdamjee House, I.I.Chundrigar Road, Karachi,\n\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a0\u00a

In [ ]:
insLLM = InsuranceERPLLM()
insLLM.store_convo_in_vector_store(f"{dummy_convo}")
insLLM.search_convo_vector_store()
insLLM.query_from_final_results(insLLM.convo_query_results)